In [1]:
# In this jupyter notebook, we will run some analysis for data in the manual inhibiton paradigm, collected online.
# We will first load the data from a json file and transform it into a pandas data frame. 
# Next, we will make sure to reformat that data frame such that every row is one trial (the json file will give us
# one row for every event)

In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = {}
exp_data =  open('jatos_results1')
for sub, jf in enumerate(exp_data):
    jd = json.loads(jf)
    df = pd.DataFrame(jd)
    #print(df)
    df['subject'] = sub

    try:
        data = pd.concat([data,df], axis = 0)
        print('fail')
    except:
        data = df
#data.reset_index(drop = True, inplace = True)
data

,success,test_part,scrWidth,scrHeight,trial_type,trial_index,time_elapsed,internal_node_id,view_history,rt,...,key_press,position,shift,inwards,flash,touchX,touchY,full_rt,response,subject
0,True,welcome,1024.0,1366.0,fullscreen,0,2175,0.0-0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,intro,NaN,NaN,instructions,1,2892,0.0-1.0,"[{""page_index"":0,""viewing_time"":710}]",711.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,NaN,intro,NaN,NaN,instructions,2,3882,0.0-2.0,"[{""page_index"":0,""viewing_time"":789}]",789.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,NaN,prefixation,NaN,NaN,html-button-response-touchdown,3,5417,0.0-3.0-0.0,NaN,530.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,NaN,fixation,NaN,NaN,html-keyboard-response,4,5596,0.0-3.0-1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,NaN,decision,NaN,NaN,html-button-response-touchdown,71,33734,0.0-3.0-2.11,NaN,NaN,...,NaN,r,1.0,0.0,1.0,1053.0,553.0,NaN,NaN,0
72,NaN,decision,NaN,NaN,html-button-response-touchdown,72,33809,0.0-3.0-3.11,NaN,NaN,...,NaN,r,1.0,0.0,1.0,1053.0,553.0,NaN,NaN,0
73,NaN,response,NaN,NaN,html-button-response-touchdown,73,33919,0.0-3.0-4.11,NaN,109.0,...,NaN,r,1.0,0.0,1.0,1053.0,553.0,387.0,True,0
74,NaN,feedback,NaN,NaN,html-button-response-touchdown,74,34921,0.0-3.0-5.11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
# next, we want to group all values together that belong to one trial.
# we can do that by assigning one value between the first event "prefixation" and the last event "feedback"
nr = 0
for start,end in zip(np.where(data.test_part == 'prefixation')[0],np.where(data.test_part == 'feedback')[0]):
    data.loc[start:end, 'trial_nr'] = nr
    nr +=1


In [5]:
# initialize the summary data frame
sdf = pd.DataFrame(columns = ['trial', 't_sync', 't_fixTouched', 't_pointJumped', 't_flashOn', 't_flashOff', 't_stimOff', 'rt_offline', 'rt_online', 'position', 'shift', 'inwards', 'flash', 'touchX', 'touchY', 'subject', 'response'])

In [6]:
# get all trial numbers
for trial in np.unique(data.trial_nr):
    # check if the trial number is nan 
    if not np.isnan(trial):
        # set a value for the rows of the summary df (sdf), matched with the trial number
        row = int(trial)
        # get the trial data only
        tdf = data[data.trial_nr == trial]
        # get the time when the experiment started as synchronization time
        t_sync = tdf[tdf.test_part == 'prefixation'].time_elapsed.values[0]
        # retrieve information about the experiment
        # the trial number
        sdf.loc[row,'trial'] = trial 
        # sync time
        sdf.loc[row,'t_sync'] = t_sync
        # time the fixation point was touched (should be 0)
        sdf.loc[row,'t_fixTouched'] = tdf[tdf.test_part == 'prefixation'].time_elapsed.values[0] - t_sync
        # time the point jumped
        sdf.loc[row,'t_pointJumped'] = tdf[tdf.test_part == 'fixation'].time_elapsed.values[0] - t_sync
        # time the flash appeared on the screen
        sdf.loc[row,'t_flashOn'] = tdf[tdf.test_part == 'decision'].time_elapsed.values[0] - t_sync
        # time the flash disappeared again
        sdf.loc[row,'t_flashOff'] = tdf[tdf.test_part == 'decision'].time_elapsed.values[1] - t_sync
        # time all stimuli where switched off (trial is over)
        sdf.loc[row,'t_stimOff'] = tdf[tdf.test_part == 'response'].time_elapsed.values[0] - t_sync
        
        # retrieve the time between go signal (jump) and flash offset
        t_go = sdf.loc[trial,'t_flashOff'] - sdf.loc[row,'t_pointJumped']
        # we will need the "response" line more often below, so we assign it to a variable with a shorter name
        resp = tdf[tdf.test_part == 'response']
        
        # offline computed response time (from go signal till touch)
        # we are missing response time that computes time from go signal till hand liftoff
        sdf.loc[row,'rt_offline'] = t_go + tdf[tdf.test_part == 'response'].rt.values[0]
        # online computed response time
        sdf.loc[row,'rt_online'] = resp.full_rt.values[0]
        # position of the jumped stimulus (left or right)
        sdf.loc[row,'position'] = resp.position.values[0]
        # if there was a shift or not (0/1)
        sdf.loc[row,'shift'] = resp['shift'].values[0]
        # if the shift was inwards or not (0/1/nan)
        sdf.loc[row,'inwards'] = resp.inwards.values[0]
        # if there was a visible flash or not
        sdf.loc[row,'flash'] = resp.flash.values[0]
        # the x coordinates where the screen was touched
        sdf.loc[row,'touchX'] = resp.touchX.values[0]
        # the y coordinates where the screen was touched
        sdf.loc[row,'touchY'] = resp.touchY.values[0]
        # the subject number
        sdf.loc[row,'subject'] = resp.subject.values[0]
        # if a response was given or not (will be false when answer was too slow)
        sdf.loc[row,'response'] = resp.response.values[0]

    

In [8]:
sdf

,trial,t_sync,t_fixTouched,t_pointJumped,t_flashOn,t_flashOff,t_stimOff,rt_offline,rt_online,position,shift,inwards,flash,touchX,touchY,subject,response
0,0,5417,0,179,359,434,732,548,548,l,0,NaN,1,717,523,0,True
1,1,8079,0,127,332,406,510,379,379,r,0,NaN,1,715,517,0,True
2,2,10439,0,253,408,482,627,370,369,l,0,NaN,0,717,524,0,True
3,3,13174,0,178,306,381,590,407,408,r,1,1,0,697,527,0,True
4,4,15599,0,253,358,432,680,424,423,r,1,1,1,716,521,0,True
5,5,18103,0,151,203,276,579,NaN,125,l,1,0,1,714,525,0,False
6,6,20559,0,177,305,380,607,426,426,l,1,1,0,712,523,0,True
7,7,23025,0,128,307,383,608,471,472,l,1,0,0,721,523,0,True
8,8,25426,0,253,432,505,764,508,508,r,1,0,0,721,520,0,True
9,9,28085,0,101,279,354,556,453,453,l,1,1,1,715,522,0,True
